In [1]:
%matplotlib inline

# Reminder Part II: Representation Learning for Recommendation

### Similarity-based methods 

Represent item/user as a feature vector

Example: item-to-item collaborative filtering, item is a vector of co-occurence with other items

### Dimensionality reduction methods

Represent item/user as a vector in shared latent space

Example: user-item matrix factorization

# Part III: Representation Learning by Deep Learning

___

Deep learning methods learn hierarchical distributed representations from the data. They have been very successful in solving perception problems such as vision, nlp, speech recognition. Increasingly, deep learning is applied to other problems, such as search and recommendation. 


![LeNet](fig/lenet.png)
![Surf](fig/surf.png)

# Part III: Overview

### Section 1: Introduction to deep learning

### Section 2: Multi-layer neural networks

* Practical exercise: deep networks for recommendation

### Section 3: Item and user representations

* Learning item representation with Convolutional Neural Networks
* Practical exercise: adding movie poster in deep network for recommendation
* Learning user representation with Recurrent Neural Networks

# Section 1: Introduction to deep learning

## Logistic regression

Logistic regression maps _input_ $x$ to _binary target_ $y$ using _weights_ vector $w$ and _bias_ $b$:

$\hat{y} = \frac{1}{1+\exp(-(xw + b))}$ 

$x \in R^{d_{in}}, y \in (0,1), w \in R^{d_{in}}, b \in R^{1}$.

## Recommendation case

**Input**, $x$, is a vector of features that describes user and candidate item

For example, user favorite movie genres, candidate movie genre, candidate movie release date.

**Target** $y$, is whether user will take action on this candidate item.

For example, watch entire movie, only watch a trailer, not watch at all.


## Generalized linear model

GLM is an extention of linear model to handle other types of $y$ that can be binary-valued, real-valued, categorical-valued:

 1. Compute pre-activations: $a(x) = xW + b$

 2. Apply **activation function** $o$ to result of 1. 

$x \in R^{d_{in}}, y \in R^{d_{out}}, W \in R^{d_{in}\times d_{out}}, b \in R^{d_{out}}$.

#### Activation functions

In the case of logistic regression, it's a sigmoid activation function: $o(x) = sigmoid(a(x)) = \frac{1}{1+\exp(-a(x))}$

Other activation functions: softmax, tanh, ReLU.

![Linear regression](fig/1_neuron_explained.png)

#### Graphical model

GLM can be seen as a neural network with a single neuron.

![Linear classifier](fig/linear_model.png)

## Representation learning

GLMs learn specific task over a feature vector describing the input data. 

It is well-known that input data representation is crucial for machine learning algorithms. Most ML systems use manually engineered features.

Although feature engineering results in performant models, hand-crafting features is time-consuming, brittle and incomplete.

Deep neural networks set an ambitious goal to **learn both specific tasks and data representation automatically**.

To learn data representation, deep networks make two assumptions. 

### Composionality hypothesis: representations are composable / form a hierarchy

We can find hierarchical structures in images, text, speech, computer programs.

![Face detection learned features](fig/face_detection_layers.png) 

### Distributional hypothesis: representations are distributed, features are not mutually exclusive

The input is represented by the activation of a set of features that are not mutually exclusive (think bit vector vs. one-hot encoded vector). It can be exponentially more efficient.

![Distributed vs local representation](fig/distributed_vs_local.png)

## Deep networks

Deep networks use GLMs as building blocks to implement the composionality property of the representations. They recursively apply the generalized linear predictors on the top of each other. 

Stacking those predictors forms multiple layers that attempt to learn representations of data with multiple levels of abstraction.

These networks are called _**multi-layer neural networks**_ or _**deep networks**_.

> Deep network is a composition of functions - matrix multiplication and non-linear transformation:

>$\mu_l(z) = f_l(zW^{(l)})$

> $DNN(x) = \mu_L \circ \mu_{L-1} \circ ... \circ \mu_0(x)$

![2 Layer NN](fig/nn_shallow_vs_deep.png)

## Advantage of deep architectures

### Modeling capacity of a single hidden layer neural network

* Universal approximation theorem (Homik, '91): single hidden layer neural network can approximate any continuous function arbitrarily well given that there is enough hidden units
* Intuition: Taylor series
* The size of hidden layer can get arbitrarily large

### Efficiency of deep architectures

* Before deep networks, there was a relative success with having a single hidden layer (Kernel machines)

$y = \sum_i \alpha_i K(X, X_i)$

$y = \mu_1 \circ \mu_0(x)$

* Depending on the kernel function, the number of terms needed to approximate a reasonable function can get arbitrarily large
* One can show that some functions that would require exponentially many hidden units in single hidden layer neural network, can be compactly represented with deep neural networks (see [Montufar, 2014](http://papers.nips.cc/paper/5422-on-the-number-of-linear-regions-of-deep-neural-networks))
* The compositional structure enables them to re-use pieces of computation exponentially often in terms of the network's depth
$y = \mu_L \circ \mu_{L-1} \circ ... \circ \mu_0(x)$
* Think time-space tradeoff in computer programs

## Advantage of distributed representations

* Hidden units can be active at the same time
* Distributed representation is exponentially more efficient than local representation
    * In local representation (e.g. clustering), a vector of size $N$ can be represent $N$ regions    
    * In distributed representation, a vector of size $N$ can already represent $2^N$ distinct regions (assuming binary values)
* This allows to generalize non-locally to near seen regions 

![Distributed vs local representation](fig/local_vs_distributed.png)

## Advantages of deep networks

* Model complex non-linear functions efficiently: using hierarchy of layers and distributed representation
* Layers act as trainable feature creators: automatic feature engineering
* Modularity: a family of methods applicable to a broad class of problems
* Model end-to-end systems: go from image to text caption directly, without intermediate steps

## Disadvantages

* Learning deep network is tricky and involves optimizing non-convex functions
* Deep networks need more data (and time) to train increased number of parameters and parts of the hierarchy that are far away from data
* Network configuration engineering is the new feature engineering   

## Brief history

* 50s: Neural networks were invented to model the intuition of how human brain works. 
* 70s: It took two decades to discover how to efficiently train these models, algorithm called backpropagation. 
* 90s: LeCun-Net introduces convolutional neural networks. At that time, the datasets were tiny and computers were slow.
* 2012s: Deep networks made a come-back in computer vision at ImageNet competition.

# Section 2: Multi-layer neural networks

Let's take a closer look at a 2 hidden layer neural network. 

![2 layer NN](fig/2_layer_nn_explained.png)

## Model

* activation at layer 1 (hidden layer): $h^{(1)} = g(xW^{(1)} + b^{(1)})$
* activation at layer 2 (hidden layer): $h^{(2)} = g(h^{(1)}W^{(2)} + b^{(2)})$
* output layer: $\hat{y} = o(g(h^{(2)}W^{(3)} + b^{(3)}))$

### Activations

* sigmoid activation $g(a) = \frac{1}{1 + exp(-a)}$
* hyperbolic tangent $g(a) = tanh(a)$
* ReLU $g(a) = max(a, 0)$
    
#### Derivatives    
* $sigmoid'(a)=sigmoid(a)*(1-sigmoid(a))$
* $tanh'(a) = (1 - tanh(a))^2$
* $relu'(a) = 1\ if\ x \gt 0, 0\ otherwise$

![Activation functions](fig/activation_functions.png)

### Output transformations

* sigmoid $o(a) = \frac{1}{1 + exp(-a)}$ for one-class classification
* softmax $o(a) = [\frac{exp(a_1)}{\sum_n exp(a_n)},..,\frac{exp(a_N)}{\sum_n exp(a_n)}]^{T}$ for multi-class classification

## Computational graph

One can see that neural networks induce a computational graph with nodes as computing units on matrices and edges transmitting them across.
Computational graph is the main abstration used in deep learning software.

![Computational graph](fig/computational_graph.jpg)

## Prediction: forward pass

NN uses _simple_ operations: multiplication, addition, sigmoid to compute an output vector $y$ based on the input $x$. This is also called a forward pass and it is very efficient.


## Loss function

Our estimate $\hat{y} = NeuralNetwork(x, \theta)$

### Classification

NN output estimates the probability of the class $c$ given $x$: $\hat{y} = p(y=c | x)$

* negative log-likelihood (cross-entropy loss): $L_{cross-entropy}(\hat{y}, y) = \sum_i H(y^{(i)}, \hat{y^{(i)}}) = - \sum_i \sum_c y_c^{(i)} \log\hat{y_c^{(i)}}$

### Regression 

* square loss $L_{square}(\hat{y}, y) = \sum_i (y^{(i)} - \hat{y^{(i)}})^2$

# Training

We turn the problem of training NN into an optimization problem:

$\theta = \{ W^{(1)},b^{(1)}, .., W^{(L+1)},b^{(L+1)} \}$  

$arg min_{\theta} \frac{1}{N}\sum_{i} L(NeuralNetwork(x^{(i)}, \theta), y^{(i)})) + \lambda Reg(\theta)$

$L$ is a loss function.

## Stohastic Gradient Descent (SGD):

* Initialize parameters $\theta = \{ W^{(1)},b^{(1)}, .., W^{(L+1)},b^{(L+1)} \}$  
* $\alpha$ is the learning rate
* For N iterations  
  * For each training example $(x^{(t)}, y^{(t)})$ in **random** order
    * $\Delta = -\nabla_{\theta} L(NeuralNetwork(x^{(t)}, \theta), y^{(t)})) - \lambda \nabla_{\theta} Reg(\theta)$  
    * $\theta = \theta + \alpha \Delta$

## Backpropagation

To apply SGD, we need to compute the derivative of the loss function with respect to the weights efficiently. 

The main mathematical insight is the **chain rule**: $[g(f(x))]'=g'(f(x)) f'(x)$ . 

It suggests that as long as we know how to write derivatives of the individual functions, we can combine them together in a simple way and computer the derivate for the whole function.

**Backpropagation algorithm** is the efficient way to compute the derivative of loss function using chain rule. 

First, compute prediction using forward pass. Then, backpropogate errors.

In practice, software is able to automatically apply backpropagation algorithm for learning a deep network.
* [Tensorflow](https://www.tensorflow.org/versions/r0.11/tutorials/index.html)
* Torch
* Theano

## Good practices

### Initialization

* initialize weights to small random values

### Regularization

#### L2 regularization
Modify the error function to encourage small $\theta$
    
$Error = L(NeuralNetwork(x^{(t)}, \theta), y^{(t)})) + \lambda \sum_k || W^{(k)} ||_2^2$

#### Dropout

  Remove hidden units stohastically - each hidden unit is to 0 with probability 0.5
  
  Hidden units cannot co-adapt to other units
  
  ![Dropout](fig/dropout.png)
  
### Model selection

Hyperparameters: learning rate, initialization, regularization parameter, network configuration

* split the data into **training, validaton and test set**
* use validation set to select the best configuration

### Other

* Mini-batches
    * compute the gradient of the loss function over a batch of examples (32-128)
    * efficient on GPUs
* Input normalization
    * zero mean, equal variance
* Optimization algorithms
    * momentum (use previous update directions with a weight)
    * per-parameter adaptive learning rates (Agagrad, RMSprop, Adam)        
* Track gradient and weights distribution over time
* Track training and validation set performance (early stopping)

![Learning curves](fig/tensorboard_lstm.png)

[Bengio, 2013](https://arxiv.org/abs/1206.5533)

## Resources

* http://www.deeplearningbook.org/
* http://videolectures.net/deeplearning2016_montreal/
* http://info.usherbrooke.ca/hlarochelle/neural_networks/content.html
* https://www.microsoft.com/en-us/research/video/tutorial-deep-learning/

## Let's see it in practice

#### Follow instructions at this link: 

https://github.com/bandofstraycats/reco_class

#### 1. Download the SSH key.

https://goo.gl/K9x9CR

#### 2. Select AWS instance from the list:

1. ec2-34-228-197-74.compute-1.amazonaws.com
2. ec2-184-73-15-103.compute-1.amazonaws.com
3. ec2-34-227-223-19.compute-1.amazonaws.com
4. ec2-107-22-120-118.compute-1.amazonaws.com

#### 2. Connect to the AWS instance using the key.
``chmod 400 reco_class.pem
ssh -i reco_class.pem ubuntu@AWS_INSTANCE``

#### 3.  Create your working directory.
``mkdir YOUR_NAME.YOUR_SURNAME
cd YOUR_NAME.YOUR_SURNAME
git clone https://github.com/bandofstraycats/reco_class.git
cd reco_class``         
     
#### 3. Pick a random port from 6000 to 7000.

#### 4. Start jupyter server.
``jupyter notebook --no-browser --port YOUR_PORT --ip=*``

#### 5. Open notebook in your browser.
``http://AWS_INSTANCE:YOUR_PORT/?token=YOUR_TOKEN``

#### 6. Navigate to the exercise.
``DL_reco_class_exercise.ipynb``

## Tensorflow

[TensorFlow](https://tensorflow.org) is an open-source machine learning library developed by Google.

Tensorflow defines dataflow graph with nodes performing **mathematical operations (ops)** and edges transmitting **tensors** (multidimensional arrays). 

The dataflow graph represents computations that are executed within a TensorFlow session on a given device (CPUs or GPUs).

![Tensorflow graph](fig/tensorflow_graph2.png)

Tensorflow is able to automatically compute gradients and apply gradient-based optimization procedure.

### Tensorflow programming model

1. Define computational graph.
2. Start the session.
3. Feed the tensor(s) in the graph.
4. Execute operations and evaluate nodes.

## Keras

[Keras](https://keras.io/) is a high-level neural networks library, running on top of either TensorFlow or Theano.

Keras allows for easy and fast prototyping.

## Wide and Deep model for Recommendation

### Wide model
* Co-counts model, memorizes co-occurences of items
* Sparse and specific

![Wide model](fig/wide_model.png)

### Deep model
* Transitivity model, generalizes from co-occurences
* Dense and similarity-based

![Deep model](fig/deep_model.png)

### Wide and deep model
* Memorizes for specific rules and generalizes for exploration

![Wide and deep](fig/wide_and_deep.png)

[Wide & Deep Learning for Recommender Systems](https://arxiv.org/abs/1606.07792)

## Deep candidate generation

![Deep candidate generation](fig/dnn-youtube.png)

[Deep Neural Networks for YouTube Recommendations](https://research.google.com/pubs/archive/45530.pdf)

# Section 3: Learning Item and User Representation

___

# Section 3.1: Learning Item Representation

Collaborative filtering methods rely on co-occurence data that might not be available for new or niche items. The problem when no or few interaction with items are available is called _item cold-start_. This scenario is adressed by _content-based methods_ that produce recommendation based on the content of item. Indeed, in many applications, items are described by multi-modal content: image, audio, text, video.

**Convolutional Neural Networks**, a particular type of deep networks, have shown to learn useful representations for image, audio and text. ConvNets have been used for content-based recommendation with encouraging results:

* [Image-based recommendations on styles and substitutes](http://cseweb.ucsd.edu/~jmcauley/pdfs/sigir15.pdf)
* [Convolutional neural networks for sentence classification](https://arxiv.org/abs/1408.5882)
* [Deep content-based music recommendation](https://papers.nips.cc/paper/5004-deep-content-based-music-recommendation.pdf)

![VBPR](fig/vbpr.png)
![Music ConvNet](fig/cnn_music.png)

# Convolutional Neural Networks (ConvNets)

___

ConvNets are neural network architectures designed to work with images. In particular, they observe that recognizing an object in an image is invariant to its location. That means, the statistics over small patch of the image remain useful over the whole image, regardess the exact position.

ConvNets work by moving small filters across the input image. These filters are re-used for recognizing patterns throughout the entire input image. 

ConvNets have been originally designed for images, but have been shown to work well with text and audio.

![Image recognition](fig/image_recognition.png)

![Convolution operation](fig/convolution_op_gif.gif) 

[CNN paper by LeCun, '98](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf)

## Convolution

**Convolution** operation convolves **filter** with the image: computer the dot product between the filter and image pixels underneath.

![CNN1](fig/cnn_1.png)
![CNN2](fig/cnn_2.png)
![CNN3](fig/cnn_3.png)
![CNN4](fig/cnn_4.png)
![CNN5](fig/cnn_5.png)

## Convolution hyperparameters

![Convolution hyperparameters](fig/convolution_hyperparameters.png)

* input volume $W$ 
* $K$ filters, $K$-outputs = $K$ feature maps
* receptive field $F$ = filter size 
* stride $S$ = number of pixels to shift the filter (1 = same size, 2 ~ half the size) 
* padding $P$ = what you do at the end of the image (valid, zero-padding = exactly the same size)

### Example

AlexNet first layer

![AlexNet first layer](fig/alexnet_first_layer.png)

* input image size $[227, 227, 3]$
* $S=4, F=11, K=96, P=0$

Size of the output = $(W−F+2P)/S+1 = [55\times55\times96]$

Number of parameters: $K\times F\times W=96\times11\times11\times3 \approx 35 000$

## Pooling

The role of the pooling layer is to compress locally each features detector.

Pooling operation takes max (or average) of the outputs of the convolution in a small patch of each feature map. 

Hyperparameters:
* Stride
* Size of pooling filter

Advantages:
* Increase the performance
* Does not increase the number of parameteres to learn
* Increase the number of hyperparameters to tune

![Pooling](fig/pooling.png)

### Stacking convolutions

Stack convolution such that the output is of an increased depth: the spatial dimention is squeezed out, the semantic complexity is increasing.

![Stack convolutions](fig/stack_convolutions.jpg)

### AlexNet

  * Stack a pyramid of several convolutional - pooling layers with increasing the depth
  * Connect the final deep and narrow representation to a fully connected layer
  * 8 layers
  
![Alexnet features](fig/alexnet_features.png)

## Resources

* [ConvNets chapter](http://www.deeplearningbook.org/contents/convnets.html)
* [Course at Stanford](http://cs231n.github.io/convolutional-networks/)
* [C. Olah’s blog post](http://colah.github.io/posts/2014-07-Understanding-Convolutions/)
* [Visual intuition for ConvNets](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/)

## Let's see it in practice

### Adding movie poster representation to deep network for recommendation

* Download movie poster images
* Apply VGG16 network to obtain movie poster features
* Add movie poster features into deep network for recommendation

# Section 3.2: Learning User Representation

Learning representation for each userId directly is impractical for many applications because of the size of userId space. The key observation is that user is identified with a sequences of items he/she interacted with.

**Recurrent Neural Network** is a special type of deep network that is designed to model sequences. Recently, RNNs have been successfully recently applied to session-based recommendation:

[Session-based Recommendations with Recurrent Neural Networks](https://arxiv.org/abs/1511.06939)

# Recurrent Neural Networks (RNNs)

___

RNNs are designed to work with sequences, such as text and speech. Importantly, sequences can be of arbitrary length.
For example, in auto-completion task, we would like to predict the next word given previous words.

![Auto-complete](fig/auto_completion.png)

The input to the RNN at every time step is the **current word** as well as a **state vector** that represents the summary of what have happened so far / the past. This **state vector** is the encoded **memory** of the RNN.

RNNs are usually composed of input and output connection and the recurrent cell.

Unfolded RNN can be seen as very deep network in which all the layers _share the same weights_.

![CNN](fig/rnn.png)

RNN-based models has significantly improved performance in machine translation and speech recognition.

## Simple RNN 

Cell is defined as an activation over the linear combination of current input and previous state.

$h_t = ReLU(x_{t}W + h_{t-1}U + b)$

$y_t = softmax(h_tV)$

* $U$ - hidden-to-hidden weights
* $W$ - input-to-hidden weights
* $V$ - hidden-to-output weights

![Simple RNN](fig/rnn_zoom_cell.png)

### Example: character RNN

![Char RNN](fig/char_rnn_explained.png)

## Problems of training simple RNNs

#### Exploding gradients

Exploding gradients are problematic because we can make too large update steps. There is a simple solution that fixes it, called gradient clipping. It consists in clipping the norm of the gradient to fixed max value.
  
#### Vanishing gradients

Vanishing gradients prevent RNN from remembering far in the past and keeping long-term dependencies. This problem motivates the new cell achitecture -- LSTM, that augment the network with an explicit memory.

## Long Short-Term Memory (LSTM)

![LSTM](fig/lstm_cell.png)

* Internal memory $C_t$ to preserve long-term dependecies across time
* Gates $i_t, f_t, o_t$ control how much to read, write or reset the memory using continuous functions
  * $i_t$ controls how much of the current state is feed into the memory
  * $f_t$ controls how much of old memory value is reset
  * $o_t$ controls how much of memory we expose 

## Predicting the continuation of the sequence

A side-effect of being able to predict the next $x_{t+1}$ is that we can continue sequences one step at a time by sampling from the output probabilities.

![RNN sequence generation](fig/rnn_seq_generation.png)

### Procedure

* Feed a word into an RNN
* At each step, we sample $y_t$ from the output distribution and add it to the generated sequence
* Set $x_{t+1} = y_t$ (feed the prediction as the input for the next step)
* Repeat

## Resources

* [RNN chapter](http://deeplearningbook.org/contents/rnn.html)
* [Y. Bengio class](http://www.iro.umontreal.ca/~bengioy/ift6266/H16/rnn.pdf)
* [C. Olah blog post](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)